In [1]:
!pip install ngboost

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 9.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 74.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 5.6 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=bdda3c297c50b33b2d398d7714006c52b5d85c95ecef345e0aacfa9284034314
  Stored in directory: /root/.cache/pip/wheels/8b/67/f4/2caaae2146198dcb824f31a303833b07b14a5ec863fb3acd7b
Successfully built autograd-gamma
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.

In [104]:
!python --version

Python 3.11.13


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import random
import sys
import os

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor #or any model of your choice
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
#To get the root mean squared error
'''
rmse=mean_squared_error(y_true,y_pred,squared=False)
'''

# Set Theme
sns.set_theme(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=True, rc=None)
%matplotlib inline

# Set seed
np.random.seed(5)
random.seed(5)
os.environ['PYTHONHASHSEED'] = str(5)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Set Pandas Defaults
pd.set_option('display.max_columns', 2000)
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_colwidth', 2000)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/used-car-price-prediction-dataset/used_cars.csv
/kaggle/input/hackathon-qualification/Starter Notebook.ipynb
/kaggle/input/hackathon-qualification/archive/sample_submission.csv
/kaggle/input/hackathon-qualification/archive/train.csv
/kaggle/input/hackathon-qualification/archive/test.csv


# Loading Dataset

In [3]:
train_path = '/kaggle/input/hackathon-qualification/archive/train.csv'
test_path = '/kaggle/input/hackathon-qualification/archive/test.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
original = pd.read_csv('/kaggle/input/used-car-price-prediction-dataset/used_cars.csv')
ss = pd.read_csv('/kaggle/input/hackathon-qualification/archive/sample_submission.csv')

print("Train shape:", train.shape)
print("Test shape:", test.shape)

Train shape: (188533, 13)
Test shape: (125690, 12)


In [4]:
# the Sprinter 2500 is the only model in this data set with more than one brand (dodge and Mercedes-Benz)

print(original.shape)

(4009, 12)


In [5]:
# fix original price

original['price'] = original['price'].apply(lambda x: int(x.replace('$', '').replace(',', '')))

In [6]:
original[original['model']=='Sprinter 2500']

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
42,Dodge,Sprinter 2500,2007,"189,719 mi.",Diesel,3.0L V6 Cylinder Engine Diesel Fuel,5-Speed A/T,White,Gray,At least 1 accident or damage reported,Yes,14000
1175,Mercedes-Benz,Sprinter 2500,2012,"100,245 mi.",Diesel,188.0HP 3.0L V6 Cylinder Engine Diesel Fuel,A/T,White,–,At least 1 accident or damage reported,Yes,24000
1352,Mercedes-Benz,Sprinter 2500,2011,"120,000 mi.",Diesel,188.0HP 3.0L V6 Cylinder Engine Diesel Fuel,5-Speed A/T,Black,Gray,None reported,Yes,31999
1947,Mercedes-Benz,Sprinter 2500,2016,"143,290 mi.",Diesel,161.0HP 2.1L 4 Cylinder Engine Diesel Fuel,A/T,White,Black,None reported,Yes,33490
3116,Mercedes-Benz,Sprinter 2500,2016,"139,200 mi.",Diesel,161.0HP 2.1L 4 Cylinder Engine Diesel Fuel,A/T,White,Black,At least 1 accident or damage reported,Yes,22000
3348,Mercedes-Benz,Sprinter 2500,2016,"399,000 mi.",Diesel,188.0HP 3.0L V6 Cylinder Engine Diesel Fuel,5-Speed A/T,Silver,–,At least 1 accident or damage reported,Yes,24500


In [7]:
train['clean_title'].unique()

array(['Yes', nan], dtype=object)

# Exploratory Data Analysis

In [8]:
original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   int64 
dtypes: int64(2), object(10)
memory usage: 376.0+ KB


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            125690 non-null  int64 
 1   brand         125690 non-null  object
 2   model         125690 non-null  object
 3   model_year    125690 non-null  int64 
 4   milage        125690 non-null  int64 
 5   fuel_type     122307 non-null  object
 6   engine        125690 non-null  object
 7   transmission  125690 non-null  object
 8   ext_col       125690 non-null  object
 9   int_col       125690 non-null  object
 10  accident      124058 non-null  object
 11  clean_title   111451 non-null  object
dtypes: int64(3), object(9)
memory usage: 11.5+ MB


In [11]:
# drop id column

# train.drop('id', inplace=True, axis=1)
# test.drop('id', inplace=True, axis=1)

print("Train shape:", train.shape)
print("Test shape:", test.shape)

Train shape: (188533, 13)
Test shape: (125690, 12)


In [12]:
train.duplicated().sum()

0

In [13]:
# fill nan

original['fuel_type'] = original['fuel_type'].fillna('Electric')
train['fuel_type'] = train['fuel_type'].fillna('Electric')
test['fuel_type'] = test['fuel_type'].fillna('Electric')

In [14]:
print(len(original['model_year'].unique()))
original['model_year'].value_counts()

34


model_year
2022    354
2021    350
2020    322
2018    315
2019    297
2016    268
2017    259
2015    228
2023    226
2014    181
2013    158
2012    141
2011    124
2008    113
2010    100
2007     98
2005     72
2009     72
2006     66
2004     60
2003     49
2001     34
2002     32
2000     17
1999     15
1998     11
1997      9
1993      9
1996      8
1994      7
2024      6
1995      6
1974      1
1992      1
Name: count, dtype: int64

In [15]:
len(train['price'].unique())

1569

In [16]:
len(original['price'].unique())

1569

In [17]:
original[original['fuel_type']=='–'].head(45)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
130,Chrysler,Pacifica Touring,2017,"87,305 mi.",–,–,9-Speed A/T,Silver,Black,None reported,Yes,9000
257,Toyota,Land Cruiser Base,1993,"231,500 mi.",–,–,A/T,White,Gray,None reported,Yes,29500
338,Mazda,Mazda3 s Grand Touring,2016,"85,000 mi.",–,–,A/T,Black,Black,At least 1 accident or damage reported,Yes,18500
491,Chrysler,Pacifica Touring,2017,"63,000 mi.",–,–,9-Speed A/T,White,Beige,None reported,Yes,20000
692,Honda,Civic EX,1993,"58,981 mi.",–,–,A/T,White,Beige,None reported,Yes,9599
855,Ford,Bronco,1974,"6,217 mi.",–,–,–,Dark Gray Metallic,–,None reported,Yes,115000
923,Dodge,Challenger R/T,2010,"100,100 mi.",–,–,Transmission w/Dual Shift Mode,Purple,Black,None reported,Yes,17000
995,Dodge,Challenger R/T Scat Pack,2019,"43,000 mi.",–,–,6-Speed M/T,Green,Black,None reported,Yes,50000
1063,Mazda,Mazda6 i Grand Touring,2016,"83,100 mi.",–,–,A/T,White,White,None reported,Yes,17800
1083,Dodge,Challenger R/T,2010,"73,000 mi.",–,–,A/T,Black,Black,None reported,Yes,17500


In [18]:
# # fix values for fuel_type == '–'
# print(original.shape)
# print(original['fuel_type'].unique())

# a = original[original['fuel_type']=='–'].copy()
# b = original[original['fuel_type']!='–'].copy()

# # manually label to the correct fuel type for '–' values
# correct = ['Gasoline', 'Diesel', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Diesel', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'Gasoline', 'E85 Flex Fuel', 'Gasoline', 'Gasoline', 'E85 Flex Fuel', 'Gasoline', 'Gasoline', 'Diesel', 'Diesel']
# print(len(correct))
# a['fuel_type'] = correct

# # join back
# c = pd.concat([a,b])
# c.reset_index(drop=True, inplace=True)
# original = c.copy()
# print(original.shape)

# original['fuel_type'].value_counts()

In [19]:
original.isna().sum()

brand             0
model             0
model_year        0
milage            0
fuel_type         0
engine            0
transmission      0
ext_col           0
int_col           0
accident        113
clean_title     596
price             0
dtype: int64

In [20]:
train.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capability,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [21]:
train['brand'].value_counts().shape

(57,)

In [22]:
original['brand'].value_counts().shape

(57,)

In [23]:
# usa_made = {
#     'Ford': 1,
#     'Hyundai': 1,
#     'Lexus': 1,
#     'INFINITI': 1,
#     'Audi': 0,
#     "Acura": 1,
#     'BMW': 1,
#     'Tesla': 1,
#     'Land': 1,
#     'Aston': 0,
#     'Toyota': 1,
#     'Lincoln': 1,
#     'Jaguar': 0,
#     'Mercedes-Benz': 1,
#     'Dodge': 1,
#     'Nissan': 1,
#     'Genesis': 1,
#     'Chevrolet': 1,
#     'Kia': 1,
#     'Jeep': 1,
#     'Bentley': 0,
#     'Honda': 1,
#     'Lucid': 1,
#     'MINI': 0,
#     'Porsche': 0,
#     'Hummer': 1,
#     'Chrysler': 1,
#     'Volvo': 1,
#     'Cadillac': 1,
#     'Lamborghini': 0,
#     'Maserati': 0,
#     'Volkswagen': 1,
#     'Subaru': 1,
#     'Rivian': 1,
#     'GMC': 1,
#     'RAM': 1,
#     'Alfa': 0,
#     'Ferrari': 0,
#     'Scion': 0,
#     'Mitsubishi': 0,
#     'Mazda': 1,
#     'Saturn': 1,
#     'Bugatti': 0,
#     'Polestar': 1,
#     'Rolls-Royce': 0,
#     'McLaren': 0,
#     'Buick': 1,
#     'Lotus': 0,
#     'Pontiac': 1,
#     'FIAT': 0,
#     'Karma': 1,
#     'Saab': 0,
#     'Mercury': 1,
#     'Plymouth': 1,
#     'smart': 0,
#     'Maybach': 1,
#     'Suzuki': 0,
# }

# usa_companies = {
#     'Ford': 1,
#     'Hyundai': 0,
#     'Lexus': 0,
#     'INFINITI': 0,
#     'Audi': 0,
#     'Acura': 0,
#     'BMW': 0,
#     'Tesla': 1,
#     'Land': 0,
#     'Aston': 0,
#     'Toyota': 0,
#     'Lincoln': 1,
#     'Jaguar': 0,
#     'Mercedes-Benz': 0,
#     'Dodge': 0,
#     'Nissan': 0,
#     'Genesis': 0,
#     'Chevrolet': 1,
#     'Kia': 0,
#     'Jeep': 0,
#     'Bentley': 0,
#     'Honda': 0,
#     'Lucid': 1,
#     'MINI': 0,
#     'Porsche': 0,
#     'Hummer': 1,
#     'Chrysler': 1,
#     'Volvo': 0,
#     'Cadillac': 1,
#     'Lamborghini': 0,
#     'Maserati': 0,
#     'Volkswagen': 0,
#     'Subaru': 0,
#     'Rivian': 1,
#     'GMC': 1,
#     'RAM': 0,
#     'Alfa': 0,
#     'Ferrari': 0,
#     'Scion': 0,
#     'Mitsubishi': 0,
#     'Mazda': 0,
#     'Saturn': 1,
#     'Bugatti': 0,
#     'Polestar': 0,
#     'Rolls-Royce': 0,
#     'McLaren': 0,
#     'Buick': 1,
#     'Lotus': 0,
#     'Pontiac': 1,
#     'FIAT': 0,
#     'Karma': 1,
#     'Saab': 0,
#     'Mercury': 1,
#     'Plymouth': 1,
#     'smart': 0,
#     'Maybach': 0,
#     'Suzuki': 0,
# }


# print(len(usa_made), len(usa_companies))

# train['usa_made'] = train['brand'].apply(lambda x: usa_made[x])
# test['usa_made'] = test['brand'].apply(lambda x: usa_made[x])
# original['usa_made'] = original['brand'].apply(lambda x: usa_made[x])

# train['usa_companies'] = train['brand'].apply(lambda x: usa_companies[x])
# test['usa_companies'] = test['brand'].apply(lambda x: usa_companies[x])
# original['usa_companies'] = original['brand'].apply(lambda x: usa_companies[x])

# Using both the Augumented dataset and the original dataset

# Step 1: Identifying the Car Entity

In [24]:
# # new feature car age

# train['age'] = train['model_year'].apply(lambda x: 2025 - x)
# test['age'] = test['model_year'].apply(lambda x: 2025 - x)
# original['age'] = original['model_year'].apply(lambda x: 2025 - x)

In [25]:
# # convert to model_year string

# train['model_year'] = train['model_year'].astype(str)
# test['model_year'] = test['model_year'].astype(str)
# original['model_year']= original['model_year'].astype(str)

In [26]:
# # create demand feature

# brand = list(original['brand'].unique())

# demand_original = {}
# for x in brand:
#     ans = len(original[original['brand']==x])
#     demand_original[x] = ans


# # create new functions
# train['demand'] = train['brand'].apply(lambda x: demand_original[x])
# test['demand'] = test['brand'].apply(lambda x: demand_original[x])
# original['demand'] = original['brand'].apply(lambda x: demand_original[x])

# # the most popular brand is a ford car, this tells me this dataset is an american sales data

# highest_demand = original['demand'].max()
# original[original['demand']==highest_demand].head(1)

In [27]:
# # identify cars

# train.fillna('missing', inplace=True)
# test.fillna('missing', inplace=True)
# original.fillna('missing', inplace=True)

# train['car'] = train['brand'] + ' ' + train['model'] + ' ' + train['model_year'] + ' ' + train['fuel_type'] + ' ' + train['engine'] + ' ' + train['transmission'] + ' ' + train['ext_col'] + ' ' + train['int_col'] + ' ' + train['accident'] + ' ' + train['clean_title'] 
# test['car'] = test['brand'] + ' ' + test['model'] + ' ' + test['model_year'] + ' ' + test['fuel_type'] + ' ' + test['engine'] + ' ' + test['transmission'] + ' ' + test['ext_col'] + ' ' + test['int_col'] + ' ' + test['accident'] + ' ' + test['clean_title']
# original['car'] = original['brand'] + ' ' + original['model'] + ' ' + original['model_year'] + ' ' + original['fuel_type'] + ' ' + original['engine'] + ' ' + original['transmission'] + ' ' + original['ext_col'] + ' ' + original['int_col'] + ' ' + original['accident'] + ' ' + original['clean_title']


# #number of unique entities in test dataset
# test_cars = list(test['car'].unique())
# train_cars = list(train['car'].unique())
# original_cars = list(original['car'].unique())

In [28]:
print(len(original['milage'].unique()))
print(len(train['milage'].unique()))
print(len(test['milage'].unique()))


2818
6651
5700


In [29]:
# original['car'].value_counts()

In [30]:
# original[original['car']=='RAM 1500 Laramie 2022 Gasoline 5.7L V8 16V MPFI OHV 8-Speed Automatic Diamond Black Black None reported missing']

In [31]:
original.columns

Index(['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title',
       'price'],
      dtype='object')

### Point 1

In [32]:
# #number of unique entities in test dataset
# test_cars = list(test['brand'].unique())
# train_cars = list(train['brand'].unique())
# original_cars = list(original['brand'].unique())

# print(f'There are {len(original_cars)} unique cars in this case')

# # min price per brand
# min_original = {}
# for x in original_cars:
#     ans = original[original['brand']==x]['price'].min()
#     min_original[x] = ans

# # max price per brand
# max_original = {}
# for x in original_cars:
#     ans = original[original['brand']==x]['price'].max()
#     max_original[x] = ans

# # create new functions
# train['min_brand_price'] = train['brand'].apply(lambda x: min_original[x])
# test['min_brand_price'] = test['brand'].apply(lambda x: min_original[x])
# original['min_brand_price'] = original['brand'].apply(lambda x: min_original[x])

# train['max_brand_price'] = train['brand'].apply(lambda x: max_original[x])
# test['max_brand_price'] = test['brand'].apply(lambda x: max_original[x])
# original['max_brand_price'] = original['brand'].apply(lambda x: max_original[x])

### Point 2

In [33]:
# # new feature unique car

# # train['car'] = train['brand'] + ' ' + train['model'] #+ '' + train['model_year']
# # test['car'] = test['brand'] + ' ' + test['model'] #+ '' + test['model_year']
# # original['car'] = original['brand'] + ' ' + original['model'] #+ '' + original['model_year']


# #number of unique entities in test dataset
# test_cars = list(test['model'].unique())
# train_cars = list(train['model'].unique())
# original_cars = list(original['model'].unique())

# print(f'There are {len(original_cars)} unique cars in this case')

# # min price per car
# min_original = {}
# for x in original_cars:
#     ans = original[original['model']==x]['price'].min()
#     min_original[x] = ans

# # max price per car
# max_original = {}
# for x in original_cars:
#     ans = original[original['model']==x]['price'].mean()
#     max_original[x] = ans

# def fix_min(x):
#     try:
#         ans = min_original[x]
#     except:
#         ans = None
#         return ans
#     else:
#         return ans
        
# def fix_max(x):
#     try:
#         ans = max_original[x]
#     except:
#         ans = None
#         return ans
#     else:
#         return ans

# # # create new functions
# # train['min_brand_price2'] = train['model'].apply(fix_min)
# # test['min_brand_price2'] = test['model'].apply(fix_min)
# # original['min_brand_price2'] = original['model'].apply(fix_min)

# # train['max_brand_price2'] = train['model'].apply(fix_max)
# # test['max_brand_price2'] = test['model'].apply(fix_max)
# # original['max_brand_price2'] = original['model'].apply(fix_max)

In [34]:
# # these cars do not exist
# test[~test['car'].isin(original_cars)].head()

### Point 3

In [35]:
# # new feature unique car

# train['car'] = train['brand'] + ' ' + train['model'] + '' + train['model_year']
# test['car'] = test['brand'] + ' ' + test['model'] + '' + test['model_year']
# original['car'] = original['brand'] + ' ' + original['model'] + '' + original['model_year']


# #number of unique entities in test dataset
# test_cars = list(test['car'].unique())
# train_cars = list(train['car'].unique())
# original_cars = list(original['car'].unique())

# print(f'There are {len(original_cars)} unique cars in this case')

# # min price per car
# min_original = {}
# for x in original_cars:
#     ans = original[original['car']==x]['price'].min()
#     min_original[x] = ans

# # max price per car
# max_original = {}
# for x in original_cars:
#     ans = original[original['car']==x]['price'].max()
#     max_original[x] = ans

# def fix_min(x):
#     try:
#         ans = min_original[x]
#     except:
#         ans = None
#         return ans
#     else:
#         return ans
        
# def fix_max(x):
#     try:
#         ans = max_original[x]
#     except:
#         ans = None
#         return ans
#     else:
#         return ans

# # create new functions
# train['min_brand_price3'] = train['car'].apply(fix_min)
# test['min_brand_price3'] = test['car'].apply(fix_min)
# original['min_brand_price3'] = original['car'].apply(fix_min)

# train['max_brand_price3'] = train['car'].apply(fix_max)
# test['max_brand_price3'] = test['car'].apply(fix_max)
# original['max_brand_price3'] = original['car'].apply(fix_max)

# Point 4

In [36]:
# # new feature unique car

# # train['car'] = train['brand'] + ' ' + train['model'] #+ '' + train['model_year']
# # test['car'] = test['brand'] + ' ' + test['model'] #+ '' + test['model_year']
# # original['car'] = original['brand'] + ' ' + original['model'] #+ '' + original['model_year']


# #number of unique entities in test dataset
# test_cars = list(test['model_year'].unique())
# train_cars = list(train['model_year'].unique())
# original_cars = list(original['model_year'].unique())

# print(f'There are {len(original_cars)} unique cars in this case')

# # min price per car
# min_original = {}
# for x in original_cars:
#     ans = original[original['model_year']==x]['price'].min()
#     min_original[x] = ans

# # max price per car
# max_original = {}
# for x in original_cars:
#     ans = original[original['model_year']==x]['price'].max()
#     max_original[x] = ans

# def fix_min(x):
#     try:
#         ans = min_original[x]
#     except:
#         ans = None
#         return ans
#     else:
#         return ans
        
# def fix_max(x):
#     try:
#         ans = max_original[x]
#     except:
#         ans = None
#         return ans
#     else:
#         return ans

# # create new functions
# train['min_brand_price4'] = train['model_year'].apply(fix_min)
# test['min_brand_price4'] = test['model_year'].apply(fix_min)
# original['min_brand_price4'] = original['model_year'].apply(fix_min)

# train['max_brand_price4'] = train['model_year'].apply(fix_max)
# test['max_brand_price4'] = test['model_year'].apply(fix_max)
# original['max_brand_price4'] = original['model_year'].apply(fix_max)

In [37]:
# fix accident
# train['accident'] = train['accident'].fillna('At least 1 accident or damage reported')
# test['accident'] = test['accident'].fillna('At least 1 accident or damage reported')
# original['accident'] = original['accident'].fillna('At least 1 accident or damage reported')

# train['accident'] = train['accident'].fillna('None reported')
# test['accident'] = test['accident'].fillna('None reported')
# original['accident'] = original['accident'].fillna('None reported')


train['accident_unknown'] = train['accident'].apply(lambda x: 0 if x in ['None reported', 'At least 1 accident or damage reported'] else 1)
train['accident_none'] = train['accident'].apply(lambda x: 1 if x =='None reported' else 0 )
train['accident_1_or_more'] = train['accident'].apply(lambda x: 1 if x=='At least 1 accident or damage reported' else 0)

test['accident_unknown'] = test['accident'].apply(lambda x: 0 if x in ['None reported', 'At least 1 accident or damage reported'] else 1)
test['accident_none'] = test['accident'].apply(lambda x: 1 if x =='None reported' else 0 )
test['accident_1_or_more'] = test['accident'].apply(lambda x: 1 if x=='At least 1 accident or damage reported' else 0)

original['accident_unknown'] = original['accident'].apply(lambda x: 0 if x in ['None reported', 'At least 1 accident or damage reported'] else 1)
original['accident_none'] = original['accident'].apply(lambda x: 1 if x =='None reported' else 0 )
original['accident_1_or_more'] = original['accident'].apply(lambda x: 1 if x=='At least 1 accident or damage reported' else 0)

In [38]:
# fix clean title

train['clean_title_yes'] = train['clean_title'].apply(lambda x: 1 if x=='Yes' else 0)
train['clean_title_unknown'] = train['clean_title'].apply(lambda x: 0 if x=='Yes' else 1)

test['clean_title_yes'] = test['clean_title'].apply(lambda x: 1 if x=='Yes' else 0)
test['clean_title_unknown'] = test['clean_title'].apply(lambda x: 0 if x=='Yes' else 1)

original['clean_title_yes'] = original['clean_title'].apply(lambda x: 1 if x=='Yes' else 0)
original['clean_title_unknown'] = original['clean_title'].apply(lambda x: 0 if x=='Yes' else 1)

In [39]:
original['fuel_type'].value_counts()

fuel_type
Gasoline          3309
Hybrid             194
Electric           170
E85 Flex Fuel      139
Diesel             116
–                   45
Plug-In Hybrid      34
not supported        2
Name: count, dtype: int64

In [40]:
original['fuel_type'].unique()

array(['E85 Flex Fuel', 'Gasoline', 'Hybrid', 'Electric', 'Diesel',
       'Plug-In Hybrid', '–', 'not supported'], dtype=object)

In [41]:
# fix fuel type

train['fuel_type_Gasoline'] = train['fuel_type'].apply(lambda x: 1 if x=='Gasoline' else 0)
train['fuel_type_Hybrid'] = train['fuel_type'].apply(lambda x: 1 if x =='Hybrid' else 0)
train['fuel_type_E85 Flex Fuel'] = train['fuel_type'].apply(lambda x: 1 if x =='E85 Flex Fuel' else 0)
train['fuel_type_Diesel'] = train['fuel_type'].apply(lambda x: 1 if x =='Diesel' else 0)
train['fuel_type_Plug_In_Hybrid'] = train['fuel_type'].apply(lambda x: 1 if x =='Plug-In Hybrid' else 0)
train['fuel_type_Hydrogen_Electric_Hybrid'] = train['fuel_type'].apply(lambda x: 1 if x =='not supported' else 0)
train['fuel_type_Electric'] = train['fuel_type'].apply(lambda x: 1 if x=='Electric' else 0)
train['fuel_type_-'] = train['fuel_type'].apply(lambda x: 1 if x=='–' else 0)

test['fuel_type_Gasoline'] = test['fuel_type'].apply(lambda x: 1 if x=='Gasoline' else 0)
test['fuel_type_Hybrid'] = test['fuel_type'].apply(lambda x: 1 if x =='Hybrid' else 0)
test['fuel_type_E85 Flex Fuel'] = test['fuel_type'].apply(lambda x: 1 if x =='E85 Flex Fuel' else 0)
test['fuel_type_Diesel'] = test['fuel_type'].apply(lambda x: 1 if x =='Diesel' else 0)
test['fuel_type_Plug_In_Hybrid'] = test['fuel_type'].apply(lambda x: 1 if x =='Plug-In Hybrid' else 0)
test['fuel_type_Hydrogen_Electric_Hybrid'] = test['fuel_type'].apply(lambda x: 1 if x =='not supported' else 0)
test['fuel_type_Electric'] = test['fuel_type'].apply(lambda x: 1 if x=='Electric' else 0)
test['fuel_type_-'] = test['fuel_type'].apply(lambda x: 1 if x=='–' else 0)

original['fuel_type_Gasoline'] = original['fuel_type'].apply(lambda x: 1 if x=='Gasoline' else 0)
original['fuel_type_Hybrid'] = original['fuel_type'].apply(lambda x: 1 if x =='Hybrid' else 0)
original['fuel_type_E85 Flex Fuel'] = original['fuel_type'].apply(lambda x: 1 if x =='E85 Flex Fuel' else 0)
original['fuel_type_Diesel'] = original['fuel_type'].apply(lambda x: 1 if x =='Diesel' else 0)
original['fuel_type_Plug_In_Hybrid'] = original['fuel_type'].apply(lambda x: 1 if x =='Plug-In Hybrid' else 0)
original['fuel_type_Hydrogen_Electric_Hybrid'] = original['fuel_type'].apply(lambda x: 1 if x =='not supported' else 0)
original['fuel_type_Electric'] = original['fuel_type'].apply(lambda x: 1 if x=='Electric' else 0)
original['fuel_type_-'] = original['fuel_type'].apply(lambda x: 1 if x=='–' else 0)

# train['fuel_type_Unknown'] = train['fuel_type'].apply(lambda x: 0 if x in ['E85 Flex Fuel', 'Gasoline', 'Hybrid', 'Electric', 'Diesel', 'Plug-In Hybrid', '–', 'not supported'] else 1)
# test['fuel_type_Unknown'] = test['fuel_type'].apply(lambda x: 0 if x in ['E85 Flex Fuel', 'Gasoline', 'Hybrid', 'Electric', 'Diesel', 'Plug-In Hybrid', '–', 'not supported'] else 1)
# original['fuel_type_Unknown'] = original['fuel_type'].apply(lambda x: 0 if x in ['E85 Flex Fuel', 'Gasoline', 'Hybrid', 'Electric', 'Diesel', 'Plug-In Hybrid', '–', 'not supported'] else 1)

In [42]:
# Fix Milage

original['milage'] = original['milage'].apply(lambda x: int(x[:-3].replace(',', '')))

# train['milage'] = train['milage'].apply(lambda x: x/1000000)
# test['milage'] = test['milage'].apply(lambda x: x/1000000)
# original['milage'] = original['milage'].apply(lambda x: x/1000000)

In [43]:
# # fixing transmision

# import re

# # Input set
# transmission = {
#     '1-Speed A/T', '1-Speed Automatic', '10-Speed A/T', '10-Speed Automatic',
#     '10-Speed Automatic with Overdrive', '2', '2-Speed A/T', '2-Speed Automatic',
#     '4-Speed A/T', '4-Speed Automatic', '5-Speed A/T', '5-Speed Automatic',
#     '5-Speed M/T', '6 Speed At/Mt', '6 Speed Mt', '6-Speed', '6-Speed A/T',
#     '6-Speed Automatic', '6-Speed Automatic with Auto-Shift',
#     '6-Speed Electronically Controlled Automatic with O', '6-Speed M/T',
#     '6-Speed Manual', '7-Speed', '7-Speed A/T', '7-Speed Automatic',
#     '7-Speed Automatic with Auto-Shift', '7-Speed DCT Automatic', '7-Speed M/T',
#     '7-Speed Manual', '8-SPEED A/T', '8-SPEED AT', '8-Speed A/T',
#     '8-Speed Automatic', '8-Speed Automatic with Auto-Shift', '8-Speed Manual',
#     '9-Speed A/T', '9-Speed Automatic', '9-Speed Automatic with Auto-Shift',
#     'A/T', 'Automatic', 'Automatic CVT', 'CVT Transmission', 'CVT-F', 'F',
#     'M/T', 'Manual', 'SCHEDULED FOR OR IN PRODUCTION', 'Single-Speed Fixed Gear',
#     'Transmission Overdrive Switch', 'Transmission w/Dual Shift Mode',
#     'Variable', '–'
# }

# # Noise entries to drop
# noise = {"2", "F", "–", "SCHEDULED FOR OR IN PRODUCTION",
#          "Transmission Overdrive Switch", "Transmission w/Dual Shift Mode"}

# # Function to categorize transmission type
# def categorize_transmission(label):
#     lbl = label.upper()
#     if "CVT" in lbl or "VARIABLE" in lbl:
#         return "CVT_Transmission"
#     elif "DCT" in lbl:
#         return "DCT_Transmission"
#     elif "M/T" in lbl or "MANUAL" in lbl:
#         return "Manual_Transmission"
#     elif "A/T" in lbl or "AUTOMATIC" in lbl or "AT/MT" in lbl:
#         return "Automatic_Transmission"
#     elif "SINGLE-SPEED" in lbl or "1-SPEED" in lbl:
#         return "Single-Speed_Transmission"
#     else:
#         return "Unknown_Transmission"

# # Function to extract gear count
# def extract_gears(label):
#     match = re.search(r"(\d+)[ -]?SPEED", label.upper())
#     if match:
#         return str(match.group(1))
#     if "CVT" in label.upper() or "VARIABLE" in label.upper():
#         return "Variable"
#     if "SINGLE-SPEED" in label.upper() or "FIXED GEAR" in label.upper():
#         return 'SINGLE-SPEED'
#     return "Unknown"

# train['Transmission_type'] = train['transmission'].apply(categorize_transmission)
# train['Number_of_Gears'] = train['transmission'].apply(extract_gears).astype(str)
# train['AUTO-SHIFT'] = train['transmission'].apply(lambda x: int("AUTO-SHIFT" in x.upper()))
# train['OVERDRIVE'] = train['transmission'].apply(lambda x: int("OVERDRIVE" in x.upper()))
# train['ELECTRONIC'] = train['transmission'].apply(lambda x: int("ELECTRONIC" in x.upper()))
# train['AT/MT'] = train['transmission'].apply(lambda x: int("AT/MT" in x.upper()))

# test['Transmission_type'] = test['transmission'].apply(categorize_transmission)
# test['Number_of_Gears'] = test['transmission'].apply(extract_gears).astype(str)
# test['AUTO-SHIFT'] = test['transmission'].apply(lambda x: int("AUTO-SHIFT" in x.upper()))
# test['OVERDRIVE'] = test['transmission'].apply(lambda x: int("OVERDRIVE" in x.upper()))
# test['ELECTRONIC'] = test['transmission'].apply(lambda x: int("ELECTRONIC" in x.upper()))
# test['AT/MT'] = test['transmission'].apply(lambda x: int("AT/MT" in x.upper()))

# original['Transmission_type'] = original['transmission'].apply(categorize_transmission)
# original['Number_of_Gears'] = original['transmission'].apply(extract_gears).astype(str)
# original['AUTO-SHIFT'] = original['transmission'].apply(lambda x: int("AUTO-SHIFT" in x.upper()))
# original['OVERDRIVE'] = original['transmission'].apply(lambda x: int("OVERDRIVE" in x.upper()))
# original['ELECTRONIC'] = original['transmission'].apply(lambda x: int("ELECTRONIC" in x.upper()))
# original['AT/MT'] = original['transmission'].apply(lambda x: int("AT/MT" in x.upper()))

In [44]:
# # fix engine

# import re

# def extract_horsepower(text: str):
#     match = re.search(r"(\d+(?:\.\d+)?)\s*HP", text.upper())
#     return float(match.group(1)) if match else -1

# def extract_displacement(text: str):
#     match = re.search(r"(\d+(?:\.\d+)?)\s*L", text.upper())
#     return float(match.group(1)) if match else -1

# def extract_num_cylinders(text: str):
#     match = re.search(r"(\d+)\s*CYLINDER", text.upper())
#     return float(match.group(1)) if match else -1

# def extract_engine_layout(text: str):
#     txt = text.upper()
#     if "V" in txt and re.search(r"V\d", txt):
#         return "V"
#     elif "INLINE" in txt or "STRAIGHT" in txt:
#         return "Inline"
#     elif "FLAT" in txt or "BOXER" in txt:
#         return "Flat"
#     elif "ROTARY" in txt or "WANKEL" in txt:
#         return "Rotary"
#     elif "W" in txt and re.search(r"W\d", txt):
#         return "W"
#     return 'Unknown'

# # def extract_fuel_type(text: str):
# #     txt = text.upper()
# #     if "ELECTRIC" in txt and "HYBRID" not in txt:
# #         return "Electric"
# #     elif "HYBRID" in txt:
# #         return "Hybrid"
# #     elif "DIESEL" in txt:
# #         return "Diesel"
# #     elif "FLEX" in txt or "FFV" in txt:
# #         return "E85 Flex Fuel"
# #     elif "GAS" in txt or "PETROL" in txt:
# #         return "Gasoline"
# #     return None

# def extract_flags(text: str):
#     txt = text.upper()
#     return {
#         "is_turbo": "TURBO" in txt,
#         "is_supercharged": "SUPERCHARGED" in txt,
#         "is_hybrid": "HYBRID" in txt,
#         "is_electric": "ELECTRIC" in txt and "HYBRID" not in txt,
#         "is_flex_fuel": "FLEX" in txt or "FFV" in txt
#     }



# train['Engine_horsepower'] = train['engine'].apply(extract_horsepower)
# train['Engine_displacement'] = train['engine'].apply(extract_displacement)
# train['Engine_num_cylinders'] = train['engine'].apply(extract_num_cylinders)
# train['Engine_engine_layout'] = train['engine'].apply(extract_engine_layout)
# #train['Engine_fuel_type'] = train['engine'].apply(extract_fuel_type)
# train['Engine_TURBO'] = train['engine'].apply(lambda x: int("TURBO" in x.upper()))
# train['Engine_SUPERCHARGED'] = train['engine'].apply(lambda x: int("SUPERCHARGED" in x.upper()))
# train['Engine_HYBRID'] = train['engine'].apply(lambda x: int("HYBRID" in x.upper()))
# train['Engine_ELECTRIC'] = train['engine'].apply(lambda x: 1 if "ELECTRIC" in x and "HYBRID" not in x else 0)
# train['Engine_FLEX'] = train['engine'].apply(lambda x: 1 if "FLEX" in x or "FFV" in x else 0)

# test['Engine_horsepower'] = test['engine'].apply(extract_horsepower)
# test['Engine_displacement'] = test['engine'].apply(extract_displacement)
# test['Engine_num_cylinders'] = test['engine'].apply(extract_num_cylinders)
# test['Engine_engine_layout'] = test['engine'].apply(extract_engine_layout)
# #test['Engine_fuel_type'] = test['engine'].apply(extract_fuel_type)
# test['Engine_TURBO'] = test['engine'].apply(lambda x: int("TURBO" in x.upper()))
# test['Engine_SUPERCHARGED'] = test['engine'].apply(lambda x: int("SUPERCHARGED" in x.upper()))
# test['Engine_HYBRID'] = test['engine'].apply(lambda x: int("HYBRID" in x.upper()))
# test['Engine_ELECTRIC'] = test['engine'].apply(lambda x: 1 if "ELECTRIC" in x and "HYBRID" not in x else 0)
# test['Engine_FLEX'] = test['engine'].apply(lambda x: 1 if "FLEX" in x or "FFV" in x else 0)

# original['Engine_horsepower'] = original['engine'].apply(extract_horsepower)
# original['Engine_displacement'] = original['engine'].apply(extract_displacement)
# original['Engine_num_cylinders'] = original['engine'].apply(extract_num_cylinders)
# original['Engine_engine_layout'] = original['engine'].apply(extract_engine_layout)
# #original['Engine_fuel_type'] = original['engine'].apply(extract_fuel_type)
# original['Engine_TURBO'] = original['engine'].apply(lambda x: int("TURBO" in x.upper()))
# original['Engine_SUPERCHARGED'] = original['engine'].apply(lambda x: int("SUPERCHARGED" in x.upper()))
# original['Engine_HYBRID'] = original['engine'].apply(lambda x: int("HYBRID" in x.upper()))
# original['Engine_ELECTRIC'] = original['engine'].apply(lambda x: 1 if "ELECTRIC" in x and "HYBRID" not in x else 0)
# original['Engine_FLEX'] = original['engine'].apply(lambda x: 1 if "FLEX" in x or "FFV" in x else 0)

In [45]:
# color_map = {
#     'Yellow': 'Yellow',
#     'Silver': 'Silver',
#     'Blue': 'Blue',
#     'Black': 'Black',
#     'White': 'White',
#     'Snowflake White Pearl Metallic': 'White',
#     'Gray': 'Gray',
#     'Green': 'Green',
#     'Santorini Black Metallic': 'Black',
#     'Purple': 'Other',
#     'Ebony Twilight Metallic': 'Other',
#     'Red': 'Red',
#     'Magnetite Black Metallic': 'Black',
#     'Diamond Black': 'Black',
#     'Vega Blue': 'Blue',
#     'Beige': 'Beige',
#     'Gold': 'Gold',
#     'Platinum White Pearl': 'White',
#     'Metallic': 'Other',
#     'White Frost Tri-Coat': 'White',
#     'Firecracker Red Clearcoat': 'Red',
#     'Phytonic Blue Metallic': 'Blue',
#     'Blu': 'Blue',
#     'Orange': 'Orange',
#     'Brown': 'Brown',
#     'Brilliant Silver Metallic': 'Silver',
#     'Black Raven': 'Black',
#     'Black Clearcoat': 'Black',
#     'Firenze Red': 'Red',
#     'Agate Black Metallic': 'Black',
#     'Glacial White Pearl': 'White',
#     'Majestic Plum Metallic': 'Other',
#     'designo Diamond White Metallic': 'White',
#     'Oxford White': 'White',
#     'Black Sapphire Metallic': 'Black',
#     'Mythos Black': 'Black',
#     'Granite Crystal Clearcoat Metallic': 'Other',
#     'White Diamond Tri-Coat': 'White',
#     'Magnetite Gray Metallic': 'Gray',
#     'Carpathian Grey Premium Metallic': 'Other',
#     'designo Diamond White Bright': 'White',
#     'Phantom Black Pearl Effect / Black Roof': 'Black',
#     'Nebula Gray Pearl': 'Gray',
#     'Deep Crystal Blue Mica': 'Blue',
#     'Flame Red Clearcoat': 'Red',
#     'Lunar Blue Metallic': 'Blue',
#     'Bright White Clearcoat': 'White',
#     'Rapid Red Metallic Tinted Clearcoat': 'Red',
#     'Caviar': 'Black',
#     'Dark Ash Metallic': 'Other',
#     'Velvet Red Pearlcoat': 'Red',
#     'Silver Zynith': 'Silver',
#     'Super Black': 'Black',
#     'Antimatter Blue Metallic': 'Blue',
#     'Dark Moon Blue Metallic': 'Blue',
#     'Summit White': 'White',
#     'Ebony Black': 'Black',
#     '–': 'Other',
#     'Black Cherry': 'Black',
#     'Delmonico Red Pearlcoat': 'Red',
#     'Platinum Quartz Metallic': 'Other',
#     'Ultra White': 'White',
#     'Python Green': 'Green',
#     'Garnet Red Metallic': 'Red',
#     'Snow White Pearl': 'White',
#     'Cajun Red Tintcoat': 'Red',
#     'Midnight Black Metallic': 'Black',
#     'Diamond White': 'White',
#     'Mythos Black Metallic': 'Black',
#     'Soul Red Crystal Metallic': 'Red',
#     'Atomic Silver': 'Silver',
#     'Obsidian': 'Black',
#     'Magnetic Metallic': 'Other',
#     'Twilight Blue Metallic': 'Blue',
#     'Star White': 'White',
#     'Stormy Sea': 'Other',
#     'Tango Red Metallic': 'Red',
#     'Hyper Red': 'Red',
#     'Portofino Gray': 'Gray',
#     'MANUFAKTUR Diamond White Bright': 'White',
#     'Snowflake White Pearl': 'White',
#     'Patriot Blue Pearlcoat': 'Blue',
#     'Tungsten Metallic': 'Other',
#     'Chronos Gray Metallic': 'Gray',
#     'Silver Ice Metallic': 'Silver',
#     'Daytona Gray Pearl Effect': 'Gray',
#     'Ruby Red Metallic Tinted Clearcoat': 'Red',
#     'Alpine White': 'White',
#     'Eminent White Pearl': 'White',
#     'Manhattan Noir Metallic': 'Beige',
#     'Quicksilver Metallic': 'Silver',
#     'Stellar Black Metallic': 'Black',
#     'Sparkling Silver': 'Silver',
#     'Blueprint': 'Blue',
#     'Crystal Black Silica': 'Black',
#     'Black Noir Pearl': 'Black',
#     'Arancio Borealis': 'Orange',
#     'Typhoon Gray': 'Gray',
#     'Ibis White': 'White',
#     'Graphite Grey': 'Other',
#     'Mineral White': 'White',
#     'Midnight Black': 'Black',
#     'Northsky Blue Metallic': 'Blue',
#     'Alta White': 'White',
#     'Brilliant Black': 'Black',
#     'Jet Black Mica': 'Black',
#     'Daytona Gray Pearl Effect w/ Black Roof': 'Black',
#     'Redline Red': 'Red',
#     'Glacier Silver Metallic': 'Silver',
#     'Magnetic Black': 'Black',
#     'Chronos Gray': 'Gray',
#     'Red Quartz Tintcoat': 'Red',
#     'Nero Noctis': 'Black',
#     'Firenze Red Metallic': 'Red',
#     'Iridescent Pearl Tricoat': 'Other',
#     'Twilight Black': 'Black',
#     'Radiant Red Metallic II': 'Red',
#     'Blue Metallic': 'Blue',
#     'Glacier White': 'White',
#     'Daytona Gray': 'Gray',
#     'Rosso Mars Metallic': 'Red',
#     'Wolf Gray': 'Gray',
#     'Santorin Black': 'Black',
#     'Designo Magno Matte': 'Other',
#     'Emerald Green Metallic': 'Green',
#     'Ruby Flare Pearl': 'Other',
#     'Lunar Silver Metallic': 'Silver',
#     'Eiger Grey Metallic': 'Other',
#     'Quartzite Grey Metallic': 'Other',
#     'Barcelona Red': 'Red',
#     'Beluga Black': 'Black',
#     'Matador Red Metallic': 'Red',
#     'Billet Silver Metallic Clearcoat': 'Silver',
#     'Anodized Blue Metallic': 'Blue',
#     'Black Forest Green': 'Black',
#     'Ice Silver Metallic': 'Silver',
#     'Sandstone Metallic': 'Other',
#     'Magnetic Gray Clearcoat': 'Gray',
#     'Crystal Black Pearl': 'Black',
#     'Pacific Blue Metallic': 'Blue',
#     'Stone Gray Metallic': 'Gray',
#     'Iconic Silver Metallic': 'Silver',
#     'Dark Sapphire': 'Other',
#     'Onyx': 'Black',
#     'Aventurine Green Metallic': 'Green',
#     'China Blue': 'Blue',
#     'Majestic Black Pearl': 'Black',
#     'Midnight Silver Metallic': 'Silver',
#     'Sting Gray Clearcoat': 'Gray',
#     'Glacier Blue Metallic': 'Blue',
#     'BLACK': 'Black',
#     'Chalk': 'White',
#     'Dark Matter Metallic': 'Other',
#     'Infrared Tintcoat': 'Red',
#     'Iridium Metallic': 'Other',
#     'Fuji White': 'White',
#     'Alfa White': 'White',
#     'Kodiak Brown Metallic': 'Brown',
#     'Aurora Black': 'Black',
#     'Onyx Black': 'Black',
#     'Nightfall Gray Metallic': 'Gray',
#     'Obsidian Black Metallic': 'Black',
#     'Phantom Black': 'Black',
#     'Remington Red Metallic': 'Red',
#     'designo Diamond White': 'White',
#     'Lizard Green': 'Green',
#     'Rosso Corsa': 'Red',
#     'Shadow Gray Metallic': 'Gray',
#     'Florett Silver': 'Silver',
#     'Quartz White': 'White',
#     'DB Black Clearcoat': 'Black',
#     'Yulong White': 'White',
#     'Eiger Grey': 'Other',
#     'Custom Color': 'Other',
#     'Electric Blue Metallic': 'Blue',
#     'Tempest': 'Gray',
#     'Lunar Rock': 'Gray',
#     'Mosaic Black Metallic': 'Black',
#     'Gecko Pearlcoat': 'Other',
#     'White Clearcoat': 'White',
#     'BLU ELEOS': 'Blue',
#     'Granite Crystal Metallic Clearcoat': 'Other',
#     'Rich Garnet Metallic': 'Other',
#     'Graphite Grey Metallic': 'Other',
#     'Bianco Icarus Metallic': 'White',
#     'Satin Steel Metallic': 'Other',
#     'BLUE': 'Blue',
#     'Moonlight Cloud': 'Other',
#     'Matador Red Mica': 'Red',
#     'Emin White': 'White',
#     'Machine Gray Metallic': 'Gray',
#     'White Platinum Tri-Coat Metallic': 'White',
#     'Cobra Beige Metallic': 'Beige',
#     'Cayenne Red Tintcoat': 'Red',
#     'Shoreline Blue Pearl': 'Blue',
#     'Vik Black': 'Black',
#     'Shimmering Silver': 'Silver',
#     'Bianco Monocerus': 'White',
#     'Carbonized Gray Metallic': 'Gray',
#     'Carrara White Metallic': 'White',
#     'Dark Slate Metallic': 'Other',
#     'Dark Graphite Metallic': 'Other',
#     'Sonic Silver Metallic': 'Silver',
#     'White Knuckle Clearcoat': 'White',
#     'Titanium Silver': 'Silver',
#     'Anthracite Blue Metallic': 'Blue',
#     'Black Obsidian': 'Black',
#     'Polymetal Gray Metallic': 'Gray',
#     'Orca Black Metallic': 'Black',
#     'Wind Chill Pearl': 'Other',
#     'Blue Reflex Mica': 'Blue',
#     'Dark Moss': 'Other',
#     'Selenite Grey Metallic': 'Other',
#     'Kemora Gray Metallic': 'Gray',
#     'Nightfall Mica': 'Other',
#     'Liquid Platinum': 'Other',
#     'Mountain Air Metallic': 'Other',
#     'Kinetic Blue': 'Blue',
#     'Santorini Black': 'Black',
#     'Carbon Black Metallic': 'Black',
#     'Gentian Blue Metallic': 'Blue',
#     'Red Multi': 'Red',
#     'Super White': 'White',
#     'Pearl White': 'White',
#     'Typhoon Gray Metallic': 'Gray',
#     'Navarra Blue Metallic': 'Blue',
#     'Bianco Isis': 'White',
#     'Navarra Blue': 'Blue',
#     'Volcano Grey Metallic': 'Other',
#     'Arctic Gray Metallic': 'Gray',
#     'Pure White': 'White',
#     'Baltic Gray': 'Gray',
#     'Glacier White Metallic': 'White',
#     'Frozen Dark Silver Metallic': 'Silver',
#     'Magnetic Gray Metallic': 'Gray',
#     'Gun Metallic': 'Other',
#     'Siren Red Tintcoat': 'Red',
#     'Deep Blue Metallic': 'Blue',
#     'Cirrus Silver Metallic': 'Silver',
#     'Deep Black Pearl Effect': 'Black',
#     'Granite': 'Other',
#     'Sunset Drift Chromaflair': 'Other',
#     'Oryx White Prl': 'White',
#     'Dark Gray Metallic': 'Gray',
#     'Bayside Blue': 'Blue',
#     'Pink': 'Red',
#     'Ice': 'Other',
#     'Mango Tango Pearlcoat': 'Orange',
#     'Burnished Bronze Metallic': 'Other',
#     'Verde': 'Green',
#     'Arctic White': 'White',
#     'Portofino Blue Metallic': 'Blue',
#     'Dazzling White': 'White',
#     'Nero Daytona': 'Black',
#     'Nautical Blue Pearl': 'Blue',
#     'Imperial Blue Metallic': 'Blue',
#     'Vulcano Black Metallic': 'Black',
#     'Silver Radiance': 'Silver',
#     'Hellayella Clearcoat': 'Other',
#     'Jungle Green': 'Green',
#     'C / C': 'Other',
#     'Yulong': 'White',
#     'Pristine White': 'White',
#     'Silky Silver': 'Silver',
#     'Caspian Blue': 'Blue',
#     'Sangria Red': 'Red',
#     'Donington Grey Metallic': 'Other',
#     'Apex Blue': 'Blue',
#     'Rift Metallic': 'Other',
#     'Fountain Blue': 'Blue',
#     'Balloon White': 'White',
#     'Matte White': 'White',
#     'Frozen White': 'White',
#     'Pacific Blue': 'Blue',
#     'Rosso': 'Red',
#     'Ironman Silver': 'Silver',
#     'Octane Red Pearlcoat': 'Red',
#     'Selenite Gray Metallic': 'Gray',
#     'Hydro Blue Pearlcoat': 'Blue',
#     'Ingot Silver Metallic': 'Silver',
#     'Quartz Blue Pearl': 'Blue',
#     'Lunare White Metallic': 'White',
#     'Ember Pearlcoat': 'Other',
#     'Brands Hatch Gray Metallic': 'Gray',
#     'Navarre Blue': 'Blue',
#     'Midnight Blue Metallic': 'Blue',
#     'Shadow Black': 'Black',
#     'Go Mango!': 'Orange',
#     'Maximum Steel Metallic': 'Other',
#     'Silver Flare Metallic': 'Silver',
#     'Billet Clearcoat Metallic': 'Other',
#     'Hampton Gray': 'Gray',
#     'Red Obsession': 'Red',
#     'Silver Mist': 'Silver',
#     'Scarlet Ember': 'Other',
#     'Crimson Red Tintcoat': 'Red',
#     'Tan': 'Beige',
#     'Isle of Man Green Metallic': 'Green',
#     'Crystal Black': 'Black',
#     'Glacier': 'Other',
#     'Iridium Silver Metallic': 'Silver',
#     'Bronze Dune Metallic': 'Other',
#     'Maroon': 'Red',
#     'Platinum Gray Metallic': 'Gray',
#     'Passion Red': 'Red',
#     'Silician Yellow': 'Yellow',
#     'Volcanic Orange': 'Orange',
#     'Crystal White Pearl': 'White',
#     'Reflex Silver': 'Silver',
#     'Blue Caelum': 'Blue',
#     'Thunder Gray': 'Gray',
#     'Ultra Black': 'Black',
#     'Indus Silver': 'Silver',
#     'Horizon Blue': 'Blue',
#     'Grigio Nimbus': 'Gray',
#     'Carpathian Grey': 'Other',
#     'Ametrin Metallic': 'Other',
#     'Jupiter Red': 'Red',
#     'GT SILVER': 'Silver'
# }

# train["ext_col"] = train["ext_col"].map(color_map).fillna(train["ext_col"])
# test["ext_col"] = test["ext_col"].map(color_map).fillna(test["ext_col"])
# original["ext_col"] = original["ext_col"].map(color_map).fillna(original["ext_col"])

# train["ext_col"] = train["ext_col"].apply(lambda x: x if x in ['Black', 'White'] else "Other")
# test["ext_col"] = test["ext_col"].apply(lambda x: lambda x: x if x in ['Black', 'White'] else "Other")
# original["ext_col"] = original["ext_col"].apply(lambda x: x if x in ['Black', 'White'] else "Other")


# train['ext_col_Black'] = train['ext_col'].apply(lambda x: 1 if x=='Black' else 0)
# train['ext_col_White'] = train['ext_col'].apply(lambda x: 1 if x=='White' else 0)
# train['ext_col_Other'] = train['ext_col'].apply(lambda x: 1 if x=='Other' else 0)

# test['ext_col_Black'] = test['ext_col'].apply(lambda x: 1 if x=='Black' else 0)
# test['ext_col_White'] = test['ext_col'].apply(lambda x: 1 if x=='White' else 0)
# test['ext_col_Other'] = test['ext_col'].apply(lambda x: 1 if x=='Other' else 0)

# original['ext_col_Black'] = original['ext_col'].apply(lambda x: 1 if x=='Black' else 0)
# original['ext_col_White'] = original['ext_col'].apply(lambda x: 1 if x=='White' else 0)
# original['ext_col_Other'] = original['ext_col'].apply(lambda x: 1 if x=='Other' else 0)

# train['ext_col'].value_counts()

In [46]:
# # Fix int_col
# color_map2 = {
#     'Gray': 'Gray',
#     'Beige': 'Beige',
#     'Black': 'Black',
#     '–': 'Other',
#     'Blue': 'Blue',
#     'White': 'White',
#     'Red': 'Red',
#     'Brown': 'Brown',
#     'Dark Galvanized': 'Gray',
#     'Parchment.': 'Beige',
#     'Boulder': 'Gray',
#     'Orange': 'Orange',
#     'Medium Earth Gray': 'Gray',
#     'Ebony': 'Black',
#     'Canberra Beige': 'Beige',
#     'Jet Black': 'Black',
#     'Silver': 'Silver',
#     'Light Platinum / Jet Black': 'Black',
#     'Macchiato/Magmagrey': 'Beige',
#     'Gold': 'Gold',
#     'Cloud': 'White',
#     'Rioja Red': 'Red',
#     'Global Black': 'Black',
#     'Green': 'Green',
#     'Medium Stone': 'Gray',
#     'Navy Pier': 'Blue',
#     'Dark Ash': 'Gray',
#     'BLACK': 'Black',
#     'Portland': 'Gray',
#     'Sandstone': 'Beige',
#     'Canberra Beige/Black': 'Beige',
#     'Diesel Gray / Black': 'Gray',
#     'Sarder Brown': 'Brown',
#     'Black Onyx': 'Black',
#     'White / Brown': 'White',
#     'Black/Gun Metal': 'Black',
#     'Slate': 'Gray',
#     'Satin Black': 'Black',
#     'Macchiato Beige/Black': 'Beige',
#     'Charcoal': 'Black',
#     'Black / Express Red': 'Black',
#     'Cappuccino': 'Brown',
#     'Aragon Brown': 'Brown',
#     'Parchment': 'Beige',
#     'Oyster W/Contrast': 'White',
#     'Adrenaline Red': 'Red',
#     'Ebony.': 'Black',
#     'Shara Beige': 'Beige',
#     'Graystone': 'Gray',
#     'Pearl Beige': 'Beige',
#     'Nero Ade': 'Black',
#     'Graphite': 'Gray',
#     'Tan/Ebony/Ebony': 'Beige',
#     'Charcoal Black': 'Black',
#     'Medium Ash Gray': 'Gray',
#     'Ebony Black': 'Black',
#     'Light Titanium': 'Silver',
#     'Sakhir Orange': 'Orange',
#     'Tan': 'Beige',
#     'Rock Gray': 'Gray',
#     'Brandy': 'Brown',
#     'Carbon Black': 'Black',
#     'Amber': 'Orange',
#     'Black w/Red Stitching': 'Black',
#     'Hotspur': 'Red',
#     'Chateau': 'Beige',
#     'Ice': 'White',
#     'Deep Garnet': 'Red',
#     'Blk': 'Black',
#     'Grace White': 'White',
#     'Oyster/Black': 'White',
#     'Mesa': 'Brown',
#     'Espresso': 'Brown',
#     'Black/Graphite': 'Black',
#     'Ebony / Ebony Accents': 'Black',
#     'Tan/Ebony': 'Beige',
#     'Ceramic': 'White',
#     'Medium Dark Slate': 'Gray',
#     'Graphite w/Gun Metal': 'Gray',
#     'Obsidian Black': 'Black',
#     'Cocoa / Dune': 'Brown',
#     'Roast': 'Brown',
#     'Yellow': 'Yellow',
#     'Hotspur Hide': 'Red',
#     'Gray w/Blue Bolsters': 'Gray',
#     'Chestnut': 'Brown',
#     'Saiga Beige': 'Beige',
#     'ORANGE': 'Orange',
#     'Charles Blue': 'Blue',
#     'Walnut': 'Brown',
#     'Ivory / Ebony': 'White',
#     'Caramel': 'Brown',
#     'Pimento Red w/Ebony': 'Red',
#     'Saddle Brown': 'Brown',
#     'Dark Gray': 'Gray',
#     'Silk Beige/Espresso Brown': 'Beige',
#     'Black / Brown': 'Black',
#     'Ebony/Light Oyster Stitch': 'Black',
#     'Ebony / Pimento': 'Black',
#     'Mistral Gray / Raven': 'Gray',
#     'Giallo Taurus / Nero Ade': 'Yellow',
#     'Tension': 'Blue',
#     'Medium Pewter': 'Gray',
#     'Black / Saddle': 'Black',
#     'Camel Leather': 'Beige',
#     'Black/Saddle Brown': 'Black',
#     'Macchiato': 'Beige',
#     'Anthracite': 'Black',
#     'Mocha': 'Brown',
#     'Whisper Beige': 'Beige',
#     'Titan Black / Quarzit': 'Black',
#     'Sahara Tan': 'Beige',
#     'Porpoise': 'Gray',
#     'Black/Red': 'Black',
#     'Titan Black': 'Black',
#     'AMG Black': 'Black',
#     'Deep Cypress': 'Green',
#     'Light Slate': 'Gray',
#     'Red / Black': 'Red',
#     'Beluga Hide': 'Black',
#     'Tupelo': 'Brown',
#     'Gideon': 'Brown',
#     'Rhapsody Blue': 'Blue',
#     'Medium Light Camel': 'Beige',
#     'Almond Beige': 'Beige',
#     'Black / Gray': 'Black',
#     'Nero': 'Black',
#     'Agave Green': 'Green',
#     'Deep Chestnut': 'Brown',
#     'Dark Auburn': 'Brown',
#     'Shale': 'Gray',
#     'Silk Beige/Black': 'Beige',
#     'BEIGE': 'Beige',
#     'Magma Red': 'Red',
#     'Linen': 'White',
#     'Black / Stone Grey': 'Black',
#     'Sand Beige': 'Beige',
#     'Red/Black': 'Red',
#     'Bianco Polar': 'White',
#     'Light Gray': 'Gray',
#     'Platinum': 'Silver',
#     'Sport': 'Other',
#     'Ash': 'Gray',
#     'Black / Graphite': 'Black',
#     'Nougat Brown': 'Brown',
#     'Camel': 'Beige',
#     'Mountain Brown': 'Brown',
#     'Pimento / Ebony': 'Red',
#     'Classic Red': 'Red',
#     'Sakhir Orange/Black': 'Orange',
#     'Cobalt Blue': 'Blue',
#     'Very Light Cashmere': 'Beige',
#     'Kyalami Orange': 'Orange',
#     'Orchid': 'White',
#     'Beluga': 'Black',
#     'WHITE': 'White'
# }

# train["int_col"] = train["int_col"].map(color_map2).fillna(train["int_col"])
# test["int_col"] = test["int_col"].map(color_map2).fillna(test["int_col"])
# original["int_col"] = original["int_col"].map(color_map2).fillna(original["int_col"])

# train["int_col"] = train["int_col"].apply(lambda x: x if x in ['Black', 'Beige'] else "Other")
# test["int_col"] = test["int_col"].apply(lambda x: lambda x: x if x in ['Black', 'Beige'] else "Other")
# original["int_col"] = original["int_col"].apply(lambda x: x if x in ['Black', 'Beige'] else "Other")

# train['int_col_Black'] = train['int_col'].apply(lambda x: 1 if x=='Black' else 0)
# train['int_col_Beige'] = train['int_col'].apply(lambda x: 1 if x=='Beige' else 0)
# train['int_col_Other'] = train['int_col'].apply(lambda x: 1 if x=='Other' else 0)

# test['int_col_Black'] = test['int_col'].apply(lambda x: 1 if x=='Black' else 0)
# test['int_col_Beige'] = test['int_col'].apply(lambda x: 1 if x=='Beige' else 0)
# test['int_col_Other'] = test['int_col'].apply(lambda x: 1 if x=='Other' else 0)

# original['int_col_Black'] = original['int_col'].apply(lambda x: 1 if x=='Black' else 0)
# original['int_col_Beige'] = original['int_col'].apply(lambda x: 1 if x=='Beige' else 0)
# original['int_col_Other'] = original['int_col'].apply(lambda x: 1 if x=='Other' else 0)

# train['int_col'].value_counts()

In [47]:
# # features to use dummy expansion on
# dum_features = ['ext_col', 'int_col']

# original = pd.get_dummies(original, columns=dum_features, drop_first=False, dtype=int)
# train = pd.get_dummies(train, columns=dum_features, drop_first=False, dtype=int)
# test = pd.get_dummies(test, columns=dum_features, drop_first=False, dtype=int)

In [48]:
original.head(1)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,accident_unknown,accident_none,accident_1_or_more,clean_title_yes,clean_title_unknown,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_E85 Flex Fuel,fuel_type_Diesel,fuel_type_Plug_In_Hybrid,fuel_type_Hydrogen_Electric_Hybrid,fuel_type_Electric,fuel_type_-
0,Ford,Utility Police Interceptor Base,2013,51000,E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capability,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,10300,0,0,1,1,0,0,0,1,0,0,0,0,0


In [49]:
len(original['model_year'].unique())

34

In [50]:
def encode_categorical_columns(df, df2, df3):
    #categorical_cols = ['brand', 'model', 'car', 'model_year', 'Engine_engine_layout', 'Transmission_type', 'Number_of_Gears',]
    categorical_cols = ['brand', 'model', 'engine', 'transmission', 'ext_col', 'int_col']
    
    print('Encoding variables...')

    encoders = {}  # store one encoder per column

    for col in categorical_cols:
        try:
            
            # Create a new encoder for this column
            le = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
            

            # Fit on train only
            # combine data
            data = df[col].to_list() #+ df2[col].to_list() + df3[col].to_list()
            data = list(set(data))
            le.fit(pd.DataFrame({col:data}))

            # Transform other sets
            df[[col]] = le.transform(df[[col]])
            df2[[col]] = le.transform(df2[[col]])
            df3[[col]] = le.transform(df3[[col]])

            # Save encoder for possible inverse_transform later
            encoders[col] = le

            print(f"✅ Successfully applied Ordinal Encoding to: {col}")
        except Exception as e:
            print(f"⚠️ Could not encode {col}. Error: {e}")

    return df, df2, df3, encoders

original, train, test, _  = encode_categorical_columns(original, train, test)

Encoding variables...
✅ Successfully applied Ordinal Encoding to: brand
✅ Successfully applied Ordinal Encoding to: model
✅ Successfully applied Ordinal Encoding to: engine
✅ Successfully applied Ordinal Encoding to: transmission
✅ Successfully applied Ordinal Encoding to: ext_col
✅ Successfully applied Ordinal Encoding to: int_col


In [51]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 26 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   id                                  188533 non-null  int64  
 1   brand                               188533 non-null  float64
 2   model                               188533 non-null  float64
 3   model_year                          188533 non-null  int64  
 4   milage                              188533 non-null  int64  
 5   fuel_type                           188533 non-null  object 
 6   engine                              188533 non-null  float64
 7   transmission                        188533 non-null  float64
 8   ext_col                             188533 non-null  float64
 9   int_col                             188533 non-null  float64
 10  accident                            186081 non-null  object 
 11  clean_title               

In [52]:
# train['milage'] = train['milage'].apply(lambda x: x/1000000)
# train['age'] = train['age'].apply(lambda x: x/100)
# train['demand'] = train['demand'].apply(lambda x: x/4009)
# train['Engine_horsepower'] = train['Engine_horsepower'].apply(lambda x: x/10000)
# train['Engine_displacement'] = train['Engine_displacement'].apply(lambda x: x/10)
# train['Engine_num_cylinders'] = train['Engine_num_cylinders'].apply(lambda x: x/100)

# test['milage'] = test['milage'].apply(lambda x: x/1000000)
# test['age'] = test['age'].apply(lambda x: x/100)
# test['demand'] = test['demand'].apply(lambda x: x/4009)
# test['Engine_horsepower'] = test['Engine_horsepower'].apply(lambda x: x/10000)
# test['Engine_displacement'] = test['Engine_displacement'].apply(lambda x: x/10)
# test['Engine_num_cylinders'] = test['Engine_num_cylinders'].apply(lambda x: x/100)


# original['milage'] = original['milage'].apply(lambda x: x/1000000)
# original['age'] = original['age'].apply(lambda x: x/100)
# original['demand'] = original['demand'].apply(lambda x: x/4009)
# original['Engine_horsepower'] = original['Engine_horsepower'].apply(lambda x: x/10000)
# original['Engine_displacement'] = original['Engine_displacement'].apply(lambda x: x/10)
# original['Engine_num_cylinders'] = original['Engine_num_cylinders'].apply(lambda x: x/100)



In [53]:
# # convert to model_year string

# train['model_year'] = train['model_year'].astype(int)
# test['model_year'] = test['model_year'].astype(int)
# original['model_year']= original['model_year'].astype(int)

In [54]:
train.head(1)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,accident_unknown,accident_none,accident_1_or_more,clean_title_yes,clean_title_unknown,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_E85 Flex Fuel,fuel_type_Diesel,fuel_type_Plug_In_Hybrid,fuel_type_Hydrogen_Electric_Hybrid,fuel_type_Electric,fuel_type_-
0,0,31.0,495.0,2007,213000,Gasoline,117.0,38.0,312.0,71.0,None reported,Yes,4200,0,1,0,1,0,1,0,0,0,0,0,0,0


In [55]:
# drop useless columns

train.drop(['fuel_type', 'accident', 'clean_title',], inplace=True, axis=1)
test.drop(['fuel_type', 'accident', 'clean_title',], inplace=True, axis=1)
original.drop(['fuel_type', 'accident', 'clean_title',], inplace=True, axis=1)

In [56]:
train.head(1)

,id,brand,model,model_year,milage,engine,transmission,ext_col,int_col,price,accident_unknown,accident_none,accident_1_or_more,clean_title_yes,clean_title_unknown,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_E85 Flex Fuel,fuel_type_Diesel,fuel_type_Plug_In_Hybrid,fuel_type_Hydrogen_Electric_Hybrid,fuel_type_Electric,fuel_type_-
0,0,31.0,495.0,2007,213000,117.0,38.0,312.0,71.0,4200,0,1,0,1,0,1,0,0,0,0,0,0,0


In [57]:
train.isna().sum()

id                                    0
brand                                 0
model                                 0
model_year                            0
milage                                0
engine                                0
transmission                          0
ext_col                               0
int_col                               0
price                                 0
accident_unknown                      0
accident_none                         0
accident_1_or_more                    0
clean_title_yes                       0
clean_title_unknown                   0
fuel_type_Gasoline                    0
fuel_type_Hybrid                      0
fuel_type_E85 Flex Fuel               0
fuel_type_Diesel                      0
fuel_type_Plug_In_Hybrid              0
fuel_type_Hydrogen_Electric_Hybrid    0
fuel_type_Electric                    0
fuel_type_-                           0
dtype: int64

In [58]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 23 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   id                                  188533 non-null  int64  
 1   brand                               188533 non-null  float64
 2   model                               188533 non-null  float64
 3   model_year                          188533 non-null  int64  
 4   milage                              188533 non-null  int64  
 5   engine                              188533 non-null  float64
 6   transmission                        188533 non-null  float64
 7   ext_col                             188533 non-null  float64
 8   int_col                             188533 non-null  float64
 9   price                               188533 non-null  int64  
 10  accident_unknown                    188533 non-null  int64  
 11  accident_none             

In [59]:
# # fill nan values

# def filling_missing_values_in_numerical_columns(df):
#     #numerical_cols_with_missing = ['model_year', 'milage'] # Replace with your columns

#     # Fill missing values using the median of each column
#     for col in df.columns:
#         median_value = df[col].median()
#         df[col].fillna(median_value, inplace=True)

    
#     return df


# # train.fillna(-99, inplace=True)
# # test.fillna(-99, inplace=True)
# # original.fillna(-99, inplace=True)

# train = filling_missing_values_in_numerical_columns(train)
# test = filling_missing_values_in_numerical_columns(test)
# original = filling_missing_values_in_numerical_columns(original)

In [60]:
original.corr()['price'].abs().sort_values()

fuel_type_Gasoline                    0.000405
fuel_type_Plug_In_Hybrid              0.001637
ext_col                               0.004035
fuel_type_Hydrogen_Electric_Hybrid    0.008673
fuel_type_Diesel                      0.009085
fuel_type_-                           0.011923
accident_unknown                      0.013493
fuel_type_Hybrid                      0.019693
fuel_type_Electric                    0.026364
brand                                 0.030957
model                                 0.033313
transmission                          0.036943
fuel_type_E85 Flex Fuel               0.052518
int_col                               0.064821
clean_title_yes                       0.085710
clean_title_unknown                   0.085710
accident_none                         0.105135
accident_1_or_more                    0.114088
model_year                            0.199496
engine                                0.285172
milage                                0.305528
price        

In [61]:
# train = train[['car', 'brand', 'age', 'milage', 'Engine_horsepower', 'Engine_engine_layout_W', 'AUTO-SHIFT', 'price']]
# test = test[['car', 'brand', 'age', 'milage', 'Engine_horsepower', 'Engine_engine_layout_W', 'AUTO-SHIFT']]
# original = original[['car', 'brand', 'age', 'milage', 'Engine_horsepower', 'Engine_engine_layout_W', 'AUTO-SHIFT', 'price']]

In [62]:
# pd.DataFrame(list(train['engine'].unique())).to_csv("new.csv",index=False)

In [63]:
# plt.figure(figsize=(30,30))
# sns.heatmap(original.corr(),cmap='coolwarm')

In [64]:
# def plots(columns,ncols_for_subplot, df):
#     ncols = ncols_for_subplot
#     nrows = -(-len(columns) // ncols)  # ceiling division

#     fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 5 * nrows))
#     axes = axes.flatten()

#     for i, col in enumerate(columns):
#         sns.scatterplot(x=col, y='price', data=df, ax=axes[i])
#         axes[i].set_title(f'Price vs. {col}')
#         axes[i].set_xlabel(col)
#         axes[i].set_ylabel('Price')

#     # hide unused subplots if any
#     for j in range(i + 1, len(axes)):
#         fig.delaxes(axes[j])

#     plt.tight_layout()
#     plt.show()

In [65]:
# categorical_cols = train_df.select_dtypes(include='object').columns
# numerical_cols = train_df.select_dtypes(exclude='object').columns
# def exploratory_data_analysis(df):
#     '''This function performs some preliminary EDA. You are free to add more to it to 
#        guide you in preparing your dataset for trainiing
#     '''
#     print("First 5 rows of the dataset:")
#     print(df.head())
    
#     # Get information about the dataset (data types, non-null values)
#     print("\nDataset information:")
#     df.info()
    
#     # Get descriptive statistics for numerical columns
#     print("\nDescriptive statistics for numerical columns:")
#     print(df.describe())
#     #Get descriptive statistics for categorical columns
#     print("\nDescriptive statistics for categorical columns:")
#     print(df.describe(include='object'))
#     #Checking for missing values
#     print("\nMissing values per column:")
#     print(df.isnull().sum())
#     # Visualize the distribution of the target variable (price)
#     plt.figure(figsize=(10, 6))
#     sns.histplot(df['price'], kde=True, bins=50)
#     plt.title('Distribution of Car Prices')
#     plt.xlabel('Price')
#     plt.ylabel('Frequency')
#     plt.show()
    
    
#     # Visualizing the relationship between all numerical features and price
#     # For example, 'mileage' and 'price'
#     print('Plotting numerical variables vs price')
#     numerical_plot=plots(numerical_cols,2,df)
   

In [66]:
# exploratory_data_analysis(train_df)

In [67]:
# def filling_missing_values_in_numerical_columns(df):
#     numerical_cols_with_missing = ['model_year', 'milage'] # Replace with your columns

#     # Fill missing values using the median of each column
#     for col in numerical_cols_with_missing:
#         median_value = df[col].median()
#         df[col].fillna(median_value, inplace=True)
    
#     print("DataFrame after filling numerical missing values:")
#     print(df.info())
#     return df


In [68]:
# def filling_missing_values_in_categorical_columns(df):
#         categorical_cols_with_missing = ['model', 'fuel_type'] # Replace with your columns
#         # You could fill missing values using the mode of each column. Feel free to choose your strategy
#         for col in categorical_cols_with_missing:
#             mode_value = df[col].mode()[0]
#             df[col].fillna(mode_value, inplace=True)
        
#         print("DataFrame after filling categorical missing values:")
#         print(df.info())
#         return df

In [69]:
# def encode_categorical_columns(df):
#     le = LabelEncoder()
    
   
#     #categorical_cols = df.select_dtypes(include='object').columns
#     print('Encoding variables...')
#     for col in categorical_cols:
        
#         try:
#             if df[col].isnull().any():
#                 # If so, fill them with a placeholder string 'missing' before encoding
#                 df[col].fillna('missing', inplace=True)
            
            
#             df[col] = le.fit_transform(df[col])
#             print(f"Successfully applied Label Encoding to: {col}")
#         except Exception as e:
#             print(f"Could not apply Label Encoding to {col}. Error: {e}")
#     return df

In [70]:
# def preprocessing(df):
#     '''
#     This function cleans your data for you. Feel free to tweak it to your tastes
#     It comprises filling missing data, encoding categorical variables etc
#     Perhaps feature engineering as well. Have fun!
#     Argument:
#         data (pd.DataFrame): The input DataFrame.
    
#     Returns:
#         pd.DataFrame: The preprocessed DataFrame.
#     '''
#     df=filling_missing_values_in_numerical_columns(df)
#     df=filling_missing_values_in_categorical_columns(df)
#     df=encode_categorical_columns(df)
    

#     return df

In [71]:

# train = train[['model', 'price', 'milage', 'age', 'accident_1_or_more']].copy()

# original = original[['model', 'price', 'milage', 'age', 'accident_1_or_more']].copy()

# test = test[['model', 'milage', 'age', 'accident_1_or_more']].copy()

In [72]:
X_train = original.drop(['price'], axis=1).values
y_train = original['price'].values

X_val = train.drop(['id', 'price'], axis=1).values
y_val = train['price'].values

# Try this
# X_train = train.drop('price', axis=1)
# y_train = train['price']

# X_val = original.drop('price', axis=1)
# y_val = original['price']

# # Split the data into training and testing sets (80% train, 20% test)
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nTraining set size: {len(X_train)}")
print(f"Testing set size: {len(X_val)}")

X_train.shape


Training set size: 4009
Testing set size: 188533


(4009, 21)

# Training your Model

In [73]:
# fix the price to the nearest thousand

# train['price'] = train['price'].apply(lambda x: np.round(x, decimals=-3))
# original['price'] = original['price'].apply(lambda x: np.round(x, decimals=-3))


In [74]:
# import ML libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer
from sklearn.model_selection import cross_val_score, RepeatedKFold, KFold

# Linear Models
from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso,
    ElasticNet,
    BayesianRidge,
    HuberRegressor,
    Lars,
    LassoLars,
    OrthogonalMatchingPursuit,
    PassiveAggressiveRegressor,
    Perceptron,  # technically classification, but often considered linear baseline
    RANSACRegressor,
    SGDRegressor,
    TheilSenRegressor,
    TweedieRegressor,
    QuantileRegressor
)

# Tree-based models
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    AdaBoostRegressor,
    BaggingRegressor,
    GradientBoostingRegressor,
    HistGradientBoostingRegressor,
    StackingRegressor,
    VotingRegressor
)

# Gaussian Processes
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

# Support Vector Machines
from sklearn.svm import SVR, LinearSVR, NuSVR

# Neighbors-based
from sklearn.neighbors import KNeighborsRegressor

# Neural Networks
from sklearn.neural_network import MLPRegressor

# External Gradient Boosting Libraries
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
#from imblearn.ensemble import BalancedRandomForestRegressor

# Other specialized regressors
from rgf.sklearn import RGFRegressor
from ngboost import NGBRegressor   # optional, install separately

# Hyperparameter tuning
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Skopt functions
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, VerboseCallback, DeltaXStopper
from skopt.space import Real, Categorical, Integer

# Utilities
from time import time

# Ensemble
from sklearn.ensemble import VotingRegressor

# Statsmodels for more statistical-focused models
import statsmodels.api as sm

In [75]:
original.head()

,brand,model,model_year,milage,engine,transmission,ext_col,int_col,price,accident_unknown,accident_none,accident_1_or_more,clean_title_yes,clean_title_unknown,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_E85 Flex Fuel,fuel_type_Diesel,fuel_type_Plug_In_Hybrid,fuel_type_Hydrogen_Electric_Hybrid,fuel_type_Electric,fuel_type_-
0,14.0,1743.0,2013,51000,581.0,16.0,29.0,14.0,10300,0,0,1,1,0,0,0,1,0,0,0,0,0
1,19.0,1182.0,2021,34742,566.0,32.0,185.0,71.0,38005,0,0,1,1,0,1,0,0,0,0,0,0,0
2,27.0,1325.0,2022,22372,541.0,40.0,38.0,14.0,54598,0,1,0,0,1,1,0,0,0,0,0,0,0
3,20.0,1242.0,2015,88900,724.0,23.0,29.0,14.0,15500,0,1,0,1,0,0,1,0,0,0,0,0,0
4,3.0,1225.0,2021,9835,200.0,32.0,120.0,14.0,34999,0,1,0,0,1,1,0,0,0,0,0,0,0


In [76]:
train.describe()

,id,brand,model,model_year,milage,engine,transmission,ext_col,int_col,price,accident_unknown,accident_none,accident_1_or_more,clean_title_yes,clean_title_unknown,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_E85 Flex Fuel,fuel_type_Diesel,fuel_type_Plug_In_Hybrid,fuel_type_Hydrogen_Electric_Hybrid,fuel_type_Electric,fuel_type_-
count,188533.000000,188533.000000,188533.000000,188533.000000,188533.000000,188533.000000,188533.000000,188533.000000,188533.000000,1.885330e+05,188533.000000,188533.000000,188533.000000,188533.000000,188533.000000,188533.000000,188533.000000,188533.000000,188533.000000,188533.000000,188533.000000,188533.000000,188533.000000
mean,94266.000000,23.161500,867.252030,2015.829998,65705.295174,623.737436,30.925180,157.330478,34.996696,4.387802e+04,0.013006,0.766518,0.220476,0.886391,0.113609,0.880164,0.036238,0.028674,0.020978,0.002763,0.000080,0.026961,0.004143
std,54424.933488,15.868168,521.807589,5.660967,49798.158076,284.145271,14.989879,114.585328,39.856868,7.881952e+04,0.113299,0.423047,0.414569,0.317336,0.317336,0.324770,0.186882,0.166889,0.143310,0.052496,0.008919,0.161969,0.064229
min,0.000000,0.000000,0.000000,1974.000000,100.000000,0.000000,0.000000,0.000000,0.000000,2.000000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47133.000000,9.000000,431.000000,2013.000000,24115.000000,389.000000,20.000000,29.000000,14.000000,1.700000e+04,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,94266.000000,21.000000,817.000000,2017.000000,57785.000000,640.000000,32.000000,128.000000,14.000000,3.082500e+04,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,141399.000000,36.000000,1320.000000,2020.000000,95400.000000,868.000000,38.000000,281.000000,56.000000,4.990000e+04,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,188532.000000,56.000000,1897.000000,2024.000000,405000.000000,1145.000000,61.000000,318.000000,155.000000,2.954083e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [77]:
original.columns

Index(['brand', 'model', 'model_year', 'milage', 'engine', 'transmission',
       'ext_col', 'int_col', 'price', 'accident_unknown', 'accident_none',
       'accident_1_or_more', 'clean_title_yes', 'clean_title_unknown',
       'fuel_type_Gasoline', 'fuel_type_Hybrid', 'fuel_type_E85 Flex Fuel',
       'fuel_type_Diesel', 'fuel_type_Plug_In_Hybrid',
       'fuel_type_Hydrogen_Electric_Hybrid', 'fuel_type_Electric',
       'fuel_type_-'],
      dtype='object')

In [78]:
# a = pd.concat([train, original], axis=1).reset_index(drop=False)
# a[a.duplicated(['brand', 'model', 'model_year', 'engine',
#        'transmission', 'ext_col', 'int_col', 'price', 'demand',
#        'max_brand_price2', 'accident_unknown', 'accident_none',
#        'accident_1_or_more', 'clean_title_yes', 'clean_title_unknown',
#        'fuel_type_Gasoline', 'fuel_type_Hybrid', 'fuel_type_E85 Flex Fuel',
#        'fuel_type_Diesel', 'fuel_type_Plug_In_Hybrid',
#        'fuel_type_Hydrogen_Electric_Hybrid', 'fuel_type_Electric'], keep=False)].head(10)[]

# train[train['car']==1059.0]

In [79]:
# original[original['car']==1059.0]

In [80]:
def train_with_cv(estimator, param_grid, X, y, cv_splits=5, scoring=None):
    """
    Perform K-Fold Cross Validation with hyperparameter tuning. Adjust params as you please

    Parameters:
    estimator : sklearn estimator
        The model to train (e.g., RandomForestRegressor(), LogisticRegression(), etc.)
    param_grid : dict
        Hyperparameter search space, e.g., {'n_estimators': [100, 200], 'max_depth': [5, 10]}
    X : Features
    y : Target variable
    cv_splits : int
        Number of folds for cross-validation
    scoring : str or callable
        Scoring metric (default: neg_root_mean_squared_error for regression)
    """
    # default scoring = RMSE for regression
    if scoring is None:
        scoring = make_scorer(mean_squared_error, squared=False)

    kfold = KFold(n_splits=cv_splits, shuffle=True, random_state=42)

    grid_search = GridSearchCV(
        estimator=estimator,
        param_grid=param_grid,
        cv=kfold,
        scoring=scoring,
        n_jobs=-1,
        verbose=10,
    )

    grid_search.fit(X, y)

    return grid_search.best_estimator_, grid_search.best_params_, grid_search.cv_results_

In [81]:
# model=RandomForestRegressor(random_state=42)
# #model.fit(X_train,y_train)
# param_grid = {
#     'n_estimators': [100, 200],
#     'max_depth': [5, 10, None],
#     'min_samples_split': [2, 5]
# }

# best_model, best_params, cv_results = train_with_cv(model, param_grid, X_train, y_train)

# print("Best Params:", best_params)

In [82]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from ngboost.distns import MultivariateNormal, Normal, k_categorical, Laplace, LogNormal, Poisson, Exponential#, TDistribution

best_model = NGBRegressor(random_state=42, n_estimators=20000, Dist=Exponential, early_stopping_rounds=5) # learning_rate=0.01,
#best_model = KNeighborsRegressor(n_neighbors=1569) #max_iter=10000
#best_model = KNeighborsClassifier(n_neighbors=1569) #max_iter=10000
#best_model = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=5, n_jobs=-1) #max_iter=10000



# best_model.fit(X_train,y_train, X_val=X_val, Y_val=y_val,)

# best_model.fit(X_val, y_val, X_val=X_val, Y_val=y_val,)

# best_model.fit(
#     np.vstack([X_train, X_val]), 
#     np.hstack([y_train, y_val]),
#     X_val=X_val,
#     Y_val=y_val,   
# )


full_data = pd.concat([original, train.drop('id', axis=1)], ignore_index=True).copy()
full_data = full_data.sort_values(by='price').copy()
X = full_data.drop('price', axis=1).values
y = full_data['price'].values

best_model.fit(
    X, 
    y,
    X_val=X_val,
    Y_val=y_val,   
)


y_val_pred = best_model.predict(X_val)
# val_rmse = mean_squared_error(y_val, y_val_pred, squared=False)
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

print("Validation RMSE:", val_rmse)

Using passed validation data to check for early stopping.
[iter 0] loss=11.5417 val_loss=11.5336 scale=1.0000 norm=0.8987
[iter 100] loss=11.1254 val_loss=11.1230 scale=2.0000 norm=1.3438
[iter 200] loss=10.9907 val_loss=10.9925 scale=1.0000 norm=0.6874
[iter 300] loss=10.9695 val_loss=10.9723 scale=1.0000 norm=0.7035
[iter 400] loss=10.9606 val_loss=10.9638 scale=1.0000 norm=0.7112
[iter 500] loss=10.9545 val_loss=10.9578 scale=1.0000 norm=0.7140
[iter 600] loss=10.9499 val_loss=10.9534 scale=1.0000 norm=0.7147
[iter 700] loss=10.9470 val_loss=10.9506 scale=1.0000 norm=0.7152
[iter 800] loss=10.9445 val_loss=10.9482 scale=1.0000 norm=0.7150
[iter 900] loss=10.9420 val_loss=10.9458 scale=1.0000 norm=0.7145
[iter 1000] loss=10.9398 val_loss=10.9436 scale=1.0000 norm=0.7141
[iter 1100] loss=10.9378 val_loss=10.9417 scale=1.0000 norm=0.7138
[iter 1200] loss=10.9359 val_loss=10.9398 scale=1.0000 norm=0.7133
[iter 1300] loss=10.9340 val_loss=10.9380 scale=1.0000 norm=0.7129
[iter 1400] loss

In [83]:
# a = X_val.copy()

# a['price'] = y_val
# a['pred'] = y_val_pred
# a.head()

In [84]:
# print(train.shape)
# train[train['car']!=-1]

In [85]:
# a = pd.DataFrame({'columns':X_train.columns, 'score':best_model.feature_importances_}).sort_values('score')
# print(a.shape)
# print(a['columns'][0:3].to_list())
# a

In [86]:
#final_model = RandomForestRegressor(**best_params,random_state=42)
# final_model = RandomForestRegressor(random_state=42, n_estimators=100, max_depth=5, n_jobs=-1)


# final_model.fit(
#     # X_train,y_train
#     X_val, y_val
#     # np.vstack([X_train, X_val]), 
#     # np.hstack([y_train, y_val])
# )



In [87]:
test2=pd.read_csv("/kaggle/input/hackathon-qualification/archive/test.csv")
X_test = test.drop(['id'], axis=1)
y_test_pred = best_model.predict(X_test)
test['price'] = y_test_pred

In [88]:
# def align(x):
    
#     value1 = x['max_brand_price2']
#     value2 = x['price']

#     if value2 > 50000:
#         ans = 50000
#     else:
#         ans = value2
#     return ans

# test['Final'] = test.apply(align, axis=1)
# test.head()


In [89]:
submission=pd.DataFrame({"id":test2['id'],"price":test['price']})
submission.to_csv("Submission112.csv",index=False)